In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
aapl_2016_2020_path = 'aapl_2016_2020.csv'
aapl_2021_2022_path = 'aapl_2021_2022.csv'

In [36]:
df1 = pd.read_csv(aapl_2016_2020_path, index_col=0, low_memory=False)
df2 = pd.read_csv(aapl_2021_2022_path, index_col=0, low_memory=False)

In [45]:
df1.columns

Index(['quote_readtime', 'quote_date', 'quote_time_hours', 'underlying_last',
       'expire_date', 'expire_unix', 'dte', 'c_delta', 'c_gamma', 'c_vega',
       'c_theta', 'c_rho', 'c_iv', 'c_volume', 'c_last', 'c_size', 'c_bid',
       'c_ask', 'strike', 'p_bid', 'p_ask', 'p_size', 'p_last', 'p_delta',
       'p_gamma', 'p_vega', 'p_theta', 'p_rho', 'p_iv', 'p_volume',
       'strike_distance', 'strike_distance_pct'],
      dtype='object')

In [44]:
df1.tail()[['underlying_last','dte', 'strike']]

,underlying_last,dte,strike
[QUOTE_UNIXTIME],,,
1490990400,143.67,658.04,185.0
1490990400,143.67,658.04,190.0
1490990400,143.67,658.04,195.0
1490990400,143.67,658.04,200.0
1490990400,143.67,658.04,210.0


In [37]:
df1.columns = [col.lower().replace('[', '').replace(']', '').strip() for col in df1.columns]
df2.columns = [col.lower().replace('[', '').replace(']', '').strip() for col in df2.columns]

In [38]:
df1.columns

Index(['quote_readtime', 'quote_date', 'quote_time_hours', 'underlying_last',
       'expire_date', 'expire_unix', 'dte', 'c_delta', 'c_gamma', 'c_vega',
       'c_theta', 'c_rho', 'c_iv', 'c_volume', 'c_last', 'c_size', 'c_bid',
       'c_ask', 'strike', 'p_bid', 'p_ask', 'p_size', 'p_last', 'p_delta',
       'p_gamma', 'p_vega', 'p_theta', 'p_rho', 'p_iv', 'p_volume',
       'strike_distance', 'strike_distance_pct'],
      dtype='object')

In [63]:
_df1 = df1.set_index('quote_date')[['underlying_last', 'expire_date', 'strike', 'c_bid', 'c_ask', 'p_bid', 'p_ask']]
_df2 = df2.set_index('quote_date')[['underlying_last', 'expire_date', 'strike', 'c_bid', 'c_ask', 'p_bid', 'p_ask']]

In [50]:
df = pd.concat((_df1, _df2))
df

,underlying_last,expire_date,strike,c_bid,c_ask,p_bid,p_ask
quote_date,,,,,,,
2019-01-02,157.92,2019-01-04,100.0,57.700000,58.100000,0.000000,0.020000
2019-01-02,157.92,2019-01-04,105.0,52.500000,52.960000,0.000000,0.010000
2019-01-02,157.92,2019-01-04,110.0,47.450000,47.790000,0.000000,0.010000
2019-01-02,157.92,2019-01-04,115.0,40.700000,44.640000,0.000000,0.010000
2019-01-02,157.92,2019-01-04,120.0,35.710000,39.610000,0.000000,0.010000
...,...,...,...,...,...,...,...
2022-09-30,138.09,2025-01-17,270.0,3.450000,4.100000,130.550000,132.900000
2022-09-30,138.09,2025-01-17,280.0,2.910000,3.650000,140.500000,142.900000
2022-09-30,138.09,2025-01-17,290.0,2.450000,3.150000,150.150000,152.950000


In [64]:
del _df1
del _df2

In [68]:
df['c_bid'] = df['c_bid'].map(lambda x: float(x) if x.strip() != '' else 0.0)
df['c_ask'] = df['c_ask'].map(lambda x: float(x) if x.strip() != '' else 0.0)
df['p_bid'] = df['p_bid'].map(lambda x: float(x) if x.strip() != '' else 0.0)
df['p_ask'] = df['p_ask'].map(lambda x: float(x) if x.strip() != '' else 0.0)

In [72]:
df['p_mid'] = df[['p_bid', 'p_ask']].mean(axis=1)
df['c_mid'] = df[['c_bid', 'c_ask']].mean(axis=1)

In [74]:
df.describe()

,underlying_last,strike,c_bid,c_ask,p_bid,p_ask,p_mid,c_mid
count,1.491178e+06,1.491178e+06,1.491178e+06,1.491178e+06,1.491178e+06,1.491178e+06,1.491178e+06,1.491178e+06
mean,1.791166e+02,1.685185e+02,3.515400e+01,3.584403e+01,2.465008e+01,2.520385e+01,2.492697e+01,3.549902e+01
std,7.462933e+01,9.305551e+01,4.718221e+01,4.778293e+01,3.843709e+01,3.896355e+01,3.869829e+01,4.748060e+01
min,9.034000e+01,2.500000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.326000e+02,1.050000e+02,1.100000e+00,1.230000e+00,3.000000e-01,3.800000e-01,3.450000e-01,1.170000e+00
50%,1.563400e+02,1.500000e+02,1.530000e+01,1.584000e+01,5.390000e+00,5.750000e+00,5.550000e+00,1.555000e+01
75%,1.942000e+02,2.150000e+02,5.345000e+01,5.455000e+01,3.425000e+01,3.520000e+01,3.472500e+01,5.401000e+01
max,5.061900e+02,1.000000e+03,4.291000e+02,4.333900e+02,5.190000e+02,5.229900e+02,5.209950e+02,4.312250e+02


---
# Task1

Найти локальную волатильность $\sigma_{loc}(K, T)$. Построить график $\sigma_{loc}(K, T)$ как функцию двух переменных.

$\sigma^2(K, T, S_0) = \frac{\frac{\partial C}{\partial T}}{\frac{1}{2}K^2\frac{\partial^2 C}{\partial K^2}}$

Given a complete set of European prices for all strikes and expirations, local volatilities are given uniquely by equation

In [78]:
len(df['strike'].unique())

491

In [82]:
df['expire_date'] = df['expire_date'].map(lambda x: pd.to_datetime(x))

In [84]:
df

,underlying_last,expire_date,strike,c_bid,c_ask,p_bid,p_ask,p_mid,c_mid
quote_date,,,,,,,,,
2019-01-02,157.92,2019-01-04,100.0,57.70,58.10,0.00,0.02,0.010,57.900
2019-01-02,157.92,2019-01-04,105.0,52.50,52.96,0.00,0.01,0.005,52.730
2019-01-02,157.92,2019-01-04,110.0,47.45,47.79,0.00,0.01,0.005,47.620
2019-01-02,157.92,2019-01-04,115.0,40.70,44.64,0.00,0.01,0.005,42.670
2019-01-02,157.92,2019-01-04,120.0,35.71,39.61,0.00,0.01,0.005,37.660
...,...,...,...,...,...,...,...,...,...
2022-09-30,138.09,2025-01-17,270.0,3.45,4.10,130.55,132.90,131.725,3.775
2022-09-30,138.09,2025-01-17,280.0,2.91,3.65,140.50,142.90,141.700,3.280
2022-09-30,138.09,2025-01-17,290.0,2.45,3.15,150.15,152.95,151.550,2.800
